In [ ]:
import time  # ⬅ 추가
from ultralytics import YOLO
import cv2
import numpy as np
import requests
import os
import datetime
import pyodbc
from uuid import uuid4
from sort import Sort

# YOLOv8 모델 로드
model = YOLO("yolov8s.pt")
tracker = Sort()



def save_exit_to_db(plate):
    conn = None
    try:
        conn = pyodbc.connect(conn_str)
        cursor = conn.cursor()
        cursor.execute("EXEC sp_VehicleExit @차량번호 = ?", plate)
        conn.commit()
        print("✅ 출차 저장 완료")
    except Exception as e:
        print("❌ 출차 저장 오류:", e)
    finally:
        if conn:
            conn.close()



def get_plate_text_from_ocr_api(cropped_img):
    temp_filename = f"temp_{uuid4().hex}.jpg"
    cv2.imwrite(temp_filename, cropped_img)
    try:
        with open(temp_filename, "rb") as f:
            files = {"file": (temp_filename, f, "image/jpeg")}
            response = requests.post(OCR_ENDPOINT, files=files)
        if response.status_code == 200:
            return response.json().get("ocr_results", [""])[0]
        else:
            print("❌ OCR 응답 실패:", response.status_code)
            return None
    except Exception as e:
        print("❌ OCR 요청 중 오류:", e)
        return None
    finally:
        os.remove(temp_filename)

video_path = "newenw각도1.mp4"
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
line_x = 1600

exit_count = 0
counted_ids = set()
previous_centers = {}

# ✅ 1초 간격 제어 변수
last_processed_time = 0
process_interval = 1.0  # 1초마다 처리

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    current_time = time.time()
    if current_time - last_processed_time < process_interval:
        continue  # 1초 안 지났으면 분석 생략

    last_processed_time = current_time  # 타이머 갱신

    results = model.predict(source=frame, classes=[2, 5, 7], conf=0.4)
    detections = []

    for box in results[0].boxes:
        x1, y1, x2, y2, conf = map(float, box.xyxy[0].tolist() + [box.conf[0]])
        detections.append([x1, y1, x2, y2, conf])

    dets = np.array(detections)
    tracks = tracker.update(dets) if dets.shape[0] > 0 else []

    for track in tracks:
        x1, y1, x2, y2, track_id = map(int, track)
        cy = int((y1 + y2) / 2)

        prev_x2 = previous_centers.get(track_id)
        previous_centers[track_id] = x2

        if prev_x2 is not None and prev_x2 <= line_x and x2 > line_x and track_id not in counted_ids:
            counted_ids.add(track_id)
            exit_count += 1

            cropped = frame[y1:y2, x1:x2]
            plate_text = get_plate_text_from_ocr_api(cropped)
            exit_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

            print(f"❌ 차량 {track_id} 출차 감지! 번호판: {plate_text} | 시간: {exit_time} | 총 출차 수: {exit_count}")
            save_exit_to_db(plate_text)

        cx = int((x1 + x2) / 2)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {track_id}", (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

    cv2.line(frame, (line_x, 0), (line_x, frame.shape[0]), (255, 255, 0), 3)
    cv2.putText(frame, f"Exit Total: {exit_count}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)

    resized_frame = cv2.resize(frame, (1280, 720))
    cv2.imshow("Vehicle Exit with SORT + OCR", resized_frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()



0: 640x384 1 car, 56.7ms
Speed: 2.8ms preprocess, 56.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 car, 58.3ms
Speed: 1.8ms preprocess, 58.3ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 car, 56.7ms
Speed: 1.6ms preprocess, 56.7ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 car, 64.0ms
Speed: 1.9ms preprocess, 64.0ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 car, 55.8ms
Speed: 1.3ms preprocess, 55.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 car, 57.6ms
Speed: 1.5ms preprocess, 57.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 384)
❌ 차량 3 출차 감지! 번호판: 23무0704 | 시간: 2025-06-05 17:31:18 | 총 출차 수: 1
✅ 출차 저장 완료

0: 640x384 1 car, 60.5ms
Speed: 1.7ms preprocess, 60.5ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 car, 63.0ms
Speed: 1.5ms preprocess, 63.0ms infer